Importing all important modules

In [15]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
import argparse
import os
from pathlib import Path
import torch
import numpy as np
import torch.backends.cudnn as cudnn
import random
%matplotlib inline

In [16]:
!pip install ultralytics
from ultralytics import YOLO

In [ ]:
# Activate google drive for colab dataset
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Iterating over sample videos

In [17]:
all_video_path  = list()

for i in range(1, 9):
  path = f"/content/training_videos/{i}.mp4"
  all_video_path.append(path)

# for video in all_video_path:
#   print(video)

Choose a video to train

In [18]:
serial = int(input(("Enter the serial of video to train: ")))

selected_video = ''
output = ''
if serial <= 8:
  selected_video = all_video_path[serial - 1]
  output = f'/content/trained_video/{serial}.mp4'

print(f"Video picked: {selected_video}")
print(f"Video output: {output}")

Enter the serial of video to train: 8
Video picked: /content/training_videos/8.mp4
Video output: /content/trained_video/8.mp4


In [20]:
# invoking YOLO object
model = YOLO("yolov8n.pt")

# Training the selected video
cap = cv2.VideoCapture(selected_video)

# Getting video width, height, and FPS to create the output video file
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = output
vidFormat = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, vidFormat, fps, (width, height))

# Video processing
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame) # this line let the YOLO model predict objects

    # making rectangular box like shape around predicted obejcts
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0].tolist())
        conf = result.conf[0].item()
        cls = result.cls[0].item()

        label = f"{model.names[int(cls)]} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

print(f"Training completed!!! Predicted video path is: {output_path}")


0: 640x384 7 persons, 2 cars, 4 buss, 1 truck, 131.5ms
Speed: 14.2ms preprocess, 131.5ms inference, 1190.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 8 persons, 2 cars, 5 buss, 1 truck, 6.8ms
Speed: 1.8ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 persons, 2 cars, 4 buss, 1 truck, 6.5ms
Speed: 1.8ms preprocess, 6.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 8 persons, 1 car, 4 buss, 1 truck, 6.4ms
Speed: 2.6ms preprocess, 6.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 8 persons, 3 cars, 3 buss, 1 truck, 6.5ms
Speed: 1.9ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 persons, 3 cars, 3 buss, 1 truck, 6.4ms
Speed: 2.1ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 6 persons, 3 cars, 3 buss, 1 truck, 6.5ms
Speed: 2.1ms preprocess, 6.5ms inferenc

Download the video

In [21]:
try:
  files.download(output)
except:
  print("No such predicted file in the directory")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>